In [68]:
# makning imports
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict,namedtuple
import collections
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import random

Read both the input data files and create two lists each for Words and NER tags. 
Also, create the NER tags list for start and end token. 

In [69]:
sent = open("hmm_train_sentences.txt", "r")
sent_list = []
for line in sent:
    for word in line.split():
        sent_list.append(word)
sent.close()
print(sent_list)

['Bailey', 'named', 'Australia', 'captain', 'Starc', 'player', 'of', '2015', 'World', 'Cup', 'Australia', 'won', '2003', '2007', '2015', 'Cups', 'Melbourne', 'Starc', 'Warner', 'knocks', 'etched', 'in', 'memory', '2003', 'SA', '2007', 'WI', '2015', 'Australia', 'were', 'venues', 'Starc', 'Warner', 'Melbourne', 'go', 'as', 'great', 'combination']


In [70]:
tags = open("hmm_train_ner.txt", "r")
tag_list = []
start_NER_list = []
end_NER_list = []
for line in tags:
    start_NER_list.append(line.split()[0])
    end_NER_list.append(line.split()[-1])
    for tag in line.split():
        tag_list.append(tag)
tags.close()

In [71]:
print(tag_list)
print(start_NER_list)
print(end_NER_list)

['PER', 'O', 'GEO', 'O', 'PER', 'O', 'O', 'TIM', 'O', 'O', 'GEO', 'O', 'TIM', 'TIM', 'TIM', 'O', 'GEO', 'PER', 'PER', 'O', 'O', 'O', 'O', 'TIM', 'GEO', 'TIM', 'GEO', 'TIM', 'GEO', 'O', 'O', 'PER', 'PER', 'GEO', 'O', 'O', 'O', 'O']
['PER', 'PER', 'GEO', 'GEO', 'TIM', 'PER']
['O', 'O', 'O', 'O', 'O', 'O']


Write functions for creating unigram, bigram tokens only for NER tags. 

In [72]:
def unigram_token(sequences):
    return Counter(sequences)

In [73]:
tag_ug = unigram_token(tag_list)
print(tag_ug)

Counter({'O': 18, 'GEO': 7, 'TIM': 7, 'PER': 6})


In [74]:
def bigram_token(sequences):
    d = Counter(sequences)
    return d

In [75]:
range_index = [(tag_list[i],tag_list[i+1]) for i in range(0,len(tag_list)-2,1)]
print(range_index)

[('PER', 'O'), ('O', 'GEO'), ('GEO', 'O'), ('O', 'PER'), ('PER', 'O'), ('O', 'O'), ('O', 'TIM'), ('TIM', 'O'), ('O', 'O'), ('O', 'GEO'), ('GEO', 'O'), ('O', 'TIM'), ('TIM', 'TIM'), ('TIM', 'TIM'), ('TIM', 'O'), ('O', 'GEO'), ('GEO', 'PER'), ('PER', 'PER'), ('PER', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'TIM'), ('TIM', 'GEO'), ('GEO', 'TIM'), ('TIM', 'GEO'), ('GEO', 'TIM'), ('TIM', 'GEO'), ('GEO', 'O'), ('O', 'O'), ('O', 'PER'), ('PER', 'PER'), ('PER', 'GEO'), ('GEO', 'O'), ('O', 'O'), ('O', 'O')]


In [76]:
tag_count_bg = bigram_token(range_index)
print(tag_count_bg)

Counter({('O', 'O'): 8, ('GEO', 'O'): 4, ('PER', 'O'): 3, ('O', 'GEO'): 3, ('O', 'TIM'): 3, ('TIM', 'GEO'): 3, ('O', 'PER'): 2, ('TIM', 'O'): 2, ('TIM', 'TIM'): 2, ('PER', 'PER'): 2, ('GEO', 'TIM'): 2, ('GEO', 'PER'): 1, ('PER', 'GEO'): 1})


In [77]:
def NER_word_count(tag, words):
    dDict = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tag_list, sent_list):
        dDict[tag][word] += 1
        
    return dDict


Also, create the NER tags list for start and end token. 

In [78]:
NER_words_count=NER_word_count(tag_list,sent_list)
NER_words_count

defaultdict(<function __main__.NER_word_count.<locals>.<lambda>>,
            {'GEO': defaultdict(int,
                         {'Australia': 3, 'Melbourne': 2, 'SA': 1, 'WI': 1}),
             'O': defaultdict(int,
                         {'Cup': 1,
                          'Cups': 1,
                          'World': 1,
                          'as': 1,
                          'captain': 1,
                          'combination': 1,
                          'etched': 1,
                          'go': 1,
                          'great': 1,
                          'in': 1,
                          'knocks': 1,
                          'memory': 1,
                          'named': 1,
                          'of': 1,
                          'player': 1,
                          'venues': 1,
                          'were': 1,
                          'won': 1}),
             'PER': defaultdict(int, {'Bailey': 1, 'Starc': 3, 'Warner': 2}),
             'TIM': defau

In [79]:
start_NER_count = Counter(start_NER_list)
start_NER_count

Counter({'GEO': 2, 'PER': 3, 'TIM': 1})

Create HiddenMarkovModel (from pomegranate library) to identify entities for words

In [80]:
#Build Hidden Markov Model
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

b. Calculate the start, end, and emission probabilities

In [81]:
# Calculate start probability
hmm_model.add_states()
start_prob={}
for ps in tag_list:
    start_prob[ps]=start_NER_count[ps]/tag_ug[ps]

In [82]:
print(start_prob)

{'PER': 0.5, 'O': 0.0, 'GEO': 0.2857142857142857, 'TIM': 0.14285714285714285}


In [83]:
#Arrive at Emission probabilties 
to_states = []
for NER, words in NER_words_count.items():
    total = float(sum(words.values()))
    distribution = {word: count/total for word, count in words.items()}
    NER_emissions = DiscreteDistribution(distribution)
    NER_state = State(NER_emissions, name=NER)
    to_states.append(NER_state)

In [84]:
## Calculate end probability
end_NER_count = Counter(end_NER_list)
for NER_state in to_states :
    hmm_model.add_transition(hmm_model.start,NER_state,start_prob[NER_state.name])
end_prob={}

for ps in tag_list:
    end_prob[ps]=end_NER_count[ps]/tag_ug[ps]

In [85]:
end_prob

{'GEO': 0.0, 'O': 0.3333333333333333, 'PER': 0.0, 'TIM': 0.0}

Calculate the transitional probability

In [91]:
# Add transition states to the HMM model
for NER_state in to_states :
    hmm_model.add_transition(NER_state,hmm_model.end,end_prob[NER_state.name])

In [86]:
# Get the transitional probability 
transition_prob_NER_word={}
for key in tag_count_bg.keys():
    transition_prob_NER_word[key]=tag_count_bg.get(key)/tag_ug[key[0]]

In [87]:
transition_prob_NER_word

{('GEO', 'O'): 0.5714285714285714,
 ('GEO', 'PER'): 0.14285714285714285,
 ('GEO', 'TIM'): 0.2857142857142857,
 ('O', 'GEO'): 0.16666666666666666,
 ('O', 'O'): 0.4444444444444444,
 ('O', 'PER'): 0.1111111111111111,
 ('O', 'TIM'): 0.16666666666666666,
 ('PER', 'GEO'): 0.16666666666666666,
 ('PER', 'O'): 0.5,
 ('PER', 'PER'): 0.3333333333333333,
 ('TIM', 'GEO'): 0.42857142857142855,
 ('TIM', 'O'): 0.2857142857142857,
 ('TIM', 'TIM'): 0.2857142857142857}

In [96]:
# Add transition probabilities to all POS
transition_prob_NER_word[('PER', 'TIM')]=0
transition_prob_NER_word[('GEO', 'GEO')]=0
transition_prob_NER_word[('TIM', 'PER')]=0

for NER_state in to_states :
    for next_NER_state in to_states :
        hmm_model.add_transition(NER_state,next_NER_state,transition_prob_NER_word[(NER_state.name,next_NER_state.name)])
hmm_model.bake()

d. Predict entities for this sentence, “Starc named 2015 Australia player.

In [97]:
# Decode NER for a new sentence
def NER_decoding(sentence, model):
    
    _, state_path = model.viterbi(sentence)
    return [state[1].name for state in state_path[1:-1]]

In [99]:
NER_tags = NER_decoding(("Starc", "named", "2015" ,"Australia","player"), hmm_model)

In [100]:
str(NER_tags)

"['PER', 'O', 'TIM', 'GEO', 'O']"